****Project Brief****
Problem: Choice Overload. Users waste more time searching than reading because of decision paralysis and a "winner-takes-all" market.
Solution: A context-aware engine that replaces popularity bias with situational matching (mood, time, environment e.g. bedtime, 10mins, long holiday/travelling).
Goal: Reduce decision fatigue and unlock the "Long Tail" of publishing/giving niche books the spotlight while helping readers find the perfect book for their current moment.

****Goodreads Webscraping****
Book data required 
- Genre 
- Title 
- Author
- Rating
- Rating counts 
- Description 
- Page numbers 
- ISBN
- Language 
- Published Year 
- Book Cover Image 
- Link to the book 13

****Open Library API***
Identifiers: ISBN-13
Physical Specs: Number of pages, physical dimensions, weight, and binding type (Hardcover, mass-market paperback, etc.).

Publishing Info: Publisher name, specific publication date, and series name.

Table of Contents: Often includes a full list of chapters (a feature many other APIs lack).

3. The "Author" Layer
Open Library treats authors as distinct entities with their own metadata.

Biographical Data: Full name, birth/death dates, and a biography.

Identifiers: Links to external authority files like VIAF, Wikidata, and Library of Congress ID.

Photos: Portraits of the author when available.

4. Digital & Community Data
Because Open Library is part of the Internet Archive, it includes unique "living" data:

Availability: Data on whether an eBook version is available to borrow, read online, or download.

Community Activity: User-generated Reading Logs (Want to Read, Currently Reading, Have Read), public Book Lists, and user ratings.

Revision History: Every single change made to a record is stored, meaning you can access previous "versions" of a book's data.

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [4]:
#Checking if the webscraping works 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
r = requests.get("https://www.goodreads.com/", headers=headers)
print (r.status_code)

200


soup = BeautifulSoup(r.text, 'html.parser' ) 
print (soup.prettify())

In [7]:
# only for specific gender

#genre_art = []

#for a in soup.select('a.gr-hyperlink[href="/genres/art"]'):
    #text = a.get_text(strip=True)
    
   
 #if text:
        #genre_art.append(text)

#print(genre_art)

#genre_art = []
#----------------------------
#for a in soup.select('a.gr-hyperlink[href="/genres/art"]'):
#text = a.get_text(strip=True)
#if text:
   #     genre_art.append(text)

#print(genre_art)

In [8]:
#Best Book Ever List from GoodReads 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
r = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever", headers=headers)
print (r.status_code)

200


In [9]:
#Prettyfing the Best Book Ever Page 
soup = BeautifulSoup(r.text, 'html.parser' ) 
#print (soup.prettify())

# Scrape book links from paginated list pages

In [10]:
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE = "https://www.goodreads.com"
LIST_URL = "https://www.goodreads.com/list/show/1.Best_Books_Ever"

HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

In [11]:
def get_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return BeautifulSoup(r.text, "lxml")

In [12]:
def extract_book_links_from_list(soup: BeautifulSoup):

    links = set()
    for a in soup.select("a.bookTitle[href*='/book/show/'], a[href^='/book/show/']"):
        href = a.get("href")
        if href:
            clean = href.split("?")[0]  # remove tracking params
            links.add(urljoin(BASE, clean))
    return links

book_urls = []
seen = set()


In [13]:
page = 1
while len(book_urls) < 1000:
    url = LIST_URL if page == 1 else f"{LIST_URL}?page={page}"
    print(f"Fetching list page {page}: {url}")

    soup = get_soup(url)
    page_links = extract_book_links_from_list(soup)

    # add new links
    new_links = [u for u in page_links if u not in seen]
    for u in new_links:
        seen.add(u)
        book_urls.append(u)
        if len(book_urls) >= 1000:
            break

Fetching list page 1: https://www.goodreads.com/list/show/1.Best_Books_Ever


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [18]:
 # if the page had no book links, stop (means layout changed or blocked)
if not page_links:
 print("No book links found on this page. Stopping.")
break

page += 1
time.sleep(1.5)  # be polite

print("Collected book URLs:", len(book_urls))
print("Example:", book_urls[:3])

NameError: name 'page_links' is not defined